# Files

Files are an abstraction that’s maintained by the operating system (OS). It presents a façade of names and hierarchy above a nest of raw bytes. Files also provide a layer of security. These have attached permissions that the OS enforces.

## File Formats

Storage media like hard disk drives work faster when reading or writing large blocks of data in serial. File formats are standards for working with data as an single, ordered sequence of bytes.

File formats manage trade-offs between performance, human readability, and portability. 

## Accessing Files

`std::fs::File` is the primary type for interacting with the filesystem. There are two methods available for creating a file: `open()` and `create()`. When you require more control, `std::fs::OpenOptions` is available.

## Interacting With The Filesystem In A Type-safe Manner

Rust provides type-safe variants of `str` and `String` in its standard library: `std::path::Path` and `std::path::PathBuf`. You can use these variants to unambiguously work with path separators in a cross-platform way. Path can address files, directories, and related abstractions, such as symbolic links.

In [10]:
use std::path::PathBuf;

let hello = PathBuf::from("/tmp/hello.txt");
println!("{:?}", hello.extension());

Some("txt")
